In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

dataF = yf.download("EURUSD=X", start="2025-03-19", end="2025-04-02", interval='5m')
dataF.iloc[:,:]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Datetime,,,,,
2025-03-19 00:00:00+00:00,1.094212,1.094212,1.094092,1.094212,0
2025-03-19 00:05:00+00:00,1.093733,1.093853,1.093733,1.093853,0
2025-03-19 00:10:00+00:00,1.093733,1.093733,1.093733,1.093733,0
2025-03-19 00:15:00+00:00,1.093972,1.093972,1.093733,1.093733,0
2025-03-19 00:20:00+00:00,1.094212,1.094212,1.094092,1.094092,0
...,...,...,...,...,...
2025-04-01 22:35:00+00:00,1.080497,1.080614,1.080380,1.080614,0
2025-04-01 22:40:00+00:00,1.080380,1.080497,1.080380,1.080497,0


In [11]:
grid_distance = 0.005
midprice = 1.09

In [12]:
def generate_grid(midprice, grid_distance, grid_range):
    return (np.arange(midprice-grid_range, midprice+grid_range, grid_distance))

grid = generate_grid(midprice=midprice, grid_distance=grid_distance, grid_range=0.1)
grid


array([0.99 , 0.995, 1.   , 1.005, 1.01 , 1.015, 1.02 , 1.025, 1.03 ,
       1.035, 1.04 , 1.045, 1.05 , 1.055, 1.06 , 1.065, 1.07 , 1.075,
       1.08 , 1.085, 1.09 , 1.095, 1.1  , 1.105, 1.11 , 1.115, 1.12 ,
       1.125, 1.13 , 1.135, 1.14 , 1.145, 1.15 , 1.155, 1.16 , 1.165,
       1.17 , 1.175, 1.18 , 1.185, 1.19 ])

In [13]:
signal = [0]*len(dataF)
i=0
for index, row in dataF.iterrows():
    for p in grid:
        if min(row.Low, row.High)<p and max(row.Low, row.High)>p:
            signal[i]=1
    i+=1

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [9]:
dataF["signal"]=signal
dataF[dataF["signal"]==1]

Price,Close,High,Low,Open,Volume,signal
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,
Datetime,,,,,,


In [10]:
dfpl = dataF[:].copy()
def SIGNAL():
    return dfpl.signal
dfpl['ATR'] = ta.atr(high = dfpl.High, low = dfpl.Low, close = dfpl.Close, length = 16)
dfpl.dropna(inplace=True)

AttributeError: 'DataFrame' object has no attribute 'High'

In [ ]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class MyStrat(Strategy):
    mysize = 50
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.5*grid_distance #5*self.data.ATR[-1]
        TPSLRatio = 0.5

        if self.signal1==1 and len(self.trades)<=10000:             
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=50, margin=1/1000, hedging=True, exclusive_orders=False)
stat = bt.run()
stat

In [5]:
backtesting.set_bokeh_output(notebook=False)
bt.plot(show_legend=False, plot_width=None, plot_equity=True, plot_return=False, 
plot_pl=False, plot_volume=False, plot_drawdown=False, smooth_equity=False, relative_equity=True, 
superimpose=True, resample=False, reverse_indicators=False, open_browser=True)

NameError: name 'backtesting' is not defined

In [ ]:
stat._trades.sort_values(by="EntryBar").head(20)